# Install Requirements

In [ ]:
!python -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
!pip install mamba-ssm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.2 MB/s eta 0:00:00
 

In [ ]:
!pip install causal-conv1d>=1.1.0

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00


# Dependencies

In [ ]:
import torch
import numpy as np
import json
import networkx as nx
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import os
from tqdm import tqdm

In [ ]:
from time import time

In [ ]:
import glob

# em-user Dataset

## Download Files

In [ ]:
use_gdrive=False
if use_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

  data_dir = "/content/drive/MyDrive/Subgraph_Mamba/data"
else:
  data_dir = "./data"

# Create a directory to store the downloaded files
folder_name = data_dir+ "/em_user_files"
os.makedirs(folder_name, exist_ok=True)

# Change to the directory
os.chdir(folder_name)

# List of direct download links for the files in the Dropbox folder
dropbox_links = [
    "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AAC8G8ZaykeBUArvaq6wyt06a/em_user/degree_sequence.txt?dl=1",
    "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AADyQ9oCLtpH9pmEEKgHyFcAa/em_user/edge_list.txt?dl=1",
    "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AAB41SEX2hftRuW-cZtbPS1Pa/em_user/ego_graphs.txt?dl=1",
    # "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AACTVyhujDdAyQV0nFIuCDm8a/em_user/shortest_path_matrix.npy?dl=1",
    "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AAC1SMrLr2ifNFUzgSCMlgfja/em_user/subgraphs.pth?dl=1",
    "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AAD0-ni027DwmHfkDr7A_DkMa/em_user/gin_embeddings.pth?dl=1",
    "https://www.dropbox.com/sh/zv7gw2bqzqev9yn/AAC1pP6F9Ws_-TIz7WcrhScJa/em_user/graphsaint_gcn_embeddings.pth?dl=1"
]

# Download each file
for i, link in enumerate(dropbox_links, start=1):
    file_name = link.split("/")[-1].split("?")[0]
    print(f"Downloading {file_name}...")
    !wget $link -O $file_name

# Change back to the original directory
os.chdir("..")


In [ ]:
# neighbors of each node in a dict
with open("em_user_files/ego_graphs.txt", "r") as file:
    file_contents = file.read()
    ego_graphs = json.loads(file_contents)
neighbor_dict = {int(key): value for key, value in ego_graphs.items()}

In [ ]:
# degrees of each node in a tensor
with open("em_user_files/degree_sequence.txt", "r") as file:
  file_contents=file.read()
  degree_sequence = json.loads(file_contents)
degrees = torch.zeros(len(degree_sequence))
for key, value in degree_sequence.items():
  degrees[int(key)]=value

In [ ]:
# Read the edge list from the text file
with open('em_user_files/edge_list.txt', 'r') as file:
    lines = file.readlines()

# Parse the edge list and create the edge tensor
edge_list = []
for line in lines:
    # Split the line and extract source and target node IDs
    source, target = map(int, line.strip().split())  # Assuming nodes are integers
    edge_list.append((source, target))

# Convert the edge list to a PyTorch tensor
edge_tensor = torch.tensor(edge_list).T

torch.Size([2, 4573417])

In [ ]:
edge_tensor

tensor([[    0,     0,     0,  ..., 57325, 57326, 57330],
        [   23,    34,    78,  ..., 57329, 57332, 57331]])

In [ ]:
edge_tensor.to('cuda')

tensor([[    0,     0,     0,  ..., 57325, 57326, 57330],
        [   23,    34,    78,  ..., 57329, 57332, 57331]], device='cuda:0')

In [ ]:
pretrained_node_embeds = torch.load("em_user_files/gin_embeddings.pth", torch.device('cpu'))

In [ ]:
pretrained_node_embeds.shape

torch.Size([57333, 128])

In [ ]:
embeddings = torch.cat((pretrained_node_embeds, torch.zeros(1, pretrained_node_embeds.shape[1])), axis=0)

## Read subgraph labels

In [ ]:
def read_subgraphs(sub_f, split = True):
    '''
    Read subgraphs from file

    Args
       - sub_f (str): filename where subgraphs are stored

    Return for each train, val, test split:
       - sub_G (list): list of nodes belonging to each subgraph
       - sub_G_label (list): labels for each subgraph
    '''

    # Enumerate/track labels
    label_idx = 0
    labels = {}


    # Train/Val/Test subgraphs
    train_sub_G = []
    val_sub_G = []
    test_sub_G = []

    # Train/Val/Test subgraph labels
    train_sub_G_label = []
    val_sub_G_label = []
    test_sub_G_label = []

    # Train/Val/Test masks
    train_mask = []
    val_mask = []
    test_mask = []

    multilabel = False

    # Parse data
    with open(sub_f) as fin:
        subgraph_idx = 0
        for line in fin:
            nodes = [int(n) for n in line.split("\t")[0].split("-") if n != ""]
            if len(nodes) != 0:
                if len(nodes) == 1: print(nodes)
                l = line.split("\t")[1].split("-")
                if len(l) > 1: multilabel = True
                for lab in l:
                    if lab not in labels.keys():
                        labels[lab] = label_idx
                        label_idx += 1
                if line.split("\t")[2].strip() == "train":
                    train_sub_G.append(nodes)
                    train_sub_G_label.append([labels[lab] for lab in l])
                    train_mask.append(subgraph_idx)
                elif line.split("\t")[2].strip() == "val":
                    val_sub_G.append(nodes)
                    val_sub_G_label.append([labels[lab] for lab in l])
                    val_mask.append(subgraph_idx)
                elif line.split("\t")[2].strip() == "test":
                    test_sub_G.append(nodes)
                    test_sub_G_label.append([labels[lab] for lab in l])
                    test_mask.append(subgraph_idx)
                subgraph_idx += 1
    if not multilabel:
        train_sub_G_label = torch.tensor(train_sub_G_label).long().squeeze()
        val_sub_G_label = torch.tensor(val_sub_G_label).long().squeeze()
        test_sub_G_label = torch.tensor(test_sub_G_label).long().squeeze()

    if len(val_mask) < len(test_mask):
        return train_sub_G, train_sub_G_label, test_sub_G, test_sub_G_label, val_sub_G, val_sub_G_label

    return train_sub_G, train_sub_G_label, val_sub_G, val_sub_G_label, test_sub_G, test_sub_G_label

In [ ]:
train_sub_G, train_sub_G_label, val_sub_G, val_sub_G_label, test_sub_G, test_sub_G_label = read_subgraphs("em_user_files/subgraphs.pth", split = True)

In [ ]:
len(train_sub_G[0])

61

## Dataset

In [ ]:
class SubgraphDataset(Dataset):
  def __init__(self, subgraph_list, subgraph_labels, degrees, neighbor_dict, seqlength=10_000, hops=3):
    '''
    subgraph_list: list of lists of ints
    subgraph_labels: list of ints
    degrees: tensor where degrees[node_id] is degree of that node
    neighbor_dict: dictionary mapping node_id to ids of neighbors
    seqlength: (optional) maximum sequence length. Append -1 to the start of sequences
    hops: (optional) number of hops away from subgraph to sample
    '''
    self.subgraph_list = subgraph_list
    self.subgraph_labels = subgraph_labels
    self.degrees = degrees
    self.neighbor_dict = neighbor_dict
    self.seqlength=seqlength
    self.hops=hops
  def __len__(self):
    return len(self.subgraph_list)
  def __getitem__(self, idx):
    '''
    need to rewrite this to use _get_sequence_ids

    could also get last_embedding which is 0-1 label of inclusion in the subgraph
      idea: just do 1 for the last len(subgraph_list[idx]) indices of the tensor
      idea: inside the _get_sequence_ids we can also return an inclusion vector so <- implementing this
    '''
    y = torch.Tensor(self.subgraph_labels[idx])
    sequence = self._get_sequence_ids(idx)
    padding_length = self.seqlength - len(sequence)
    padding_tensor = torch.full((padding_length,), -1)
    sequence = torch.cat((padding_tensor, sequence))
    inclusion = self._get_inclusion(idx, sequence)
    return sequence, inclusion, len(self.subgraph_list[idx]), y
  def _sort_by_degree(self, node_ids):
    '''
    node_ids - tensor of the node ids of nodes we want to sort by degree
    largest to smallest bc then we can reverse the tensor at the very last step
    '''
    degrees = self.degrees[node_ids]
    return node_ids[torch.argsort(degrees, descending=True)]

  def _get_neighbor_ids(self, sequence):
    '''
    sequence - tensor. this is the previous sequence

    Let v be a node k-1 hops away from the subgraph.
    Then the neighbors of v are sorted by degree but still kept together in the sequence.
    '''
    neighbor_lists=[]
    for s in sequence:
      if s.item() in self.neighbor_dict.keys() and s.item() not in self.current_explored:
        self.current_explored.add(s.item())
        neighbors = torch.IntTensor(self.neighbor_dict[s.item()])
        neighbor_lists.append(self._sort_by_degree(neighbors))
        self.current_seq_len+=len(neighbors)
      if self.current_seq_len >= self.seqlength:
        break
    return torch.cat(neighbor_lists)
  def _get_inclusion(self, idx, sequence):
    '''
    idx: int, index for ith subgraph in the dataset
    sequence: tensor, 1d, with node IDs
    returns: tensor with 0 and 1, same shape as sequence.

    the returned tensor has 1 if the node ID in sequence at that position is included in subgraph.
    '''
    subgraph = torch.IntTensor(self.subgraph_list[idx])
    inclusion = (sequence.unsqueeze(1) == subgraph).any(dim=1).to(torch.float)
    return inclusion
  def _get_sequence_ids(self, idx):
    '''
    idx: int, index ith subgraph in dataset
    returns 1-d tensor of length at most self.seqlength

    Generate a sequence of node IDs associated with a subgraph.
    Nodes further away from the subgraph appear earlier in the sequence.
    Nodes with the same distance from the subgraph are grouped by path to subgraph.
    Nodes within same group are sorted by degree.
    '''
    sequences = [self._sort_by_degree(torch.IntTensor(self.subgraph_list[idx]))]
    self.current_seq_len = len(sequences[0])
    self.current_explored=set()
    for i in range(self.hops):
      if self.current_seq_len < self.seqlength:
        neighbor_ids = self._get_neighbor_ids(sequences[i])
        sequences.append(neighbor_ids[~torch.isin(neighbor_ids, sequences[i])])
        self.current_seq_len=sum(len(sequences[j]) for j in range(len(sequences)))
    return torch.cat(sequences).flip(dims=[0])[-self.seqlength:]


# Model Class

In [ ]:
from mamba_ssm import Mamba
device = torch.device('cuda:0')

## Model with GCNConv

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import degree, sort_edge_index, to_dense_batch
from mamba_ssm import Mamba
from torch_geometric.nn import GCNConv


class SubgraphMamba(nn.Module):
    def __init__(self, num_features, hidden_dim, mamba_dim, num_classes, embeddings, edge_tensor, num_mamba_layers=1):
        super(SubgraphMamba, self).__init__()
        self.edge_tensor = edge_tensor
        self.embeddings = embeddings
        self.mamba = Mamba(
            d_model=mamba_dim,
            d_state=16,
            d_conv=4,
            expand=2,
        )
        self.mamba_layers=[]
        for m in range(num_mamba_layers):
          mamba=Mamba(d_model=hidden_dim, d_state=16, d_conv=4, expand=2)
          mamba.to(device)
          self.mamba_layers.append(mamba)
        # uncomment below when you have a good GPU
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)

        # self.fc1 = nn.Linear(num_features, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, sequence, inclusion, subgraph_size):
        # Preprocessing
        emb = embeddings[sequence]
        last_entry=inclusion.unsqueeze(-1)
        emb = torch.cat((emb, last_entry), dim=-1)

        # start with linear layer projection
        emb = self.conv1(emb, self.edge_tensor, None)
        emb = self.fc1(emb)

        # Mamba layers
        for mamba in self.mamba_layers:
          emb=mamba(emb)

        # a bit awkward but I don't know how else to compute my aggregation
        agg = []
        for e, size in zip(emb, subgraph_size):
          agg.append(torch.sum(e[-size:], axis=0)/size)
        emb = torch.stack(agg)

        # Linear layer for final class, assume 0-1 classification
        emb = self.fc2(emb)
        emb = F.sigmoid(emb)

        return emb

## No GCNConv

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import degree, sort_edge_index, to_dense_batch
from mamba_ssm import Mamba
from torch_geometric.nn import GCNConv


class SubgraphMamba(nn.Module):
    def __init__(self, num_features, hidden_dim, mamba_dim, num_classes, embeddings, num_mamba_layers=1):
        super(SubgraphMamba, self).__init__()
        # self.edge_tensor = edge_tensor
        self.embeddings = embeddings
        self.mamba = Mamba(
            d_model=mamba_dim,
            d_state=16,
            d_conv=4,
            expand=2,
        )
        self.mamba_layers=[]
        for m in range(num_mamba_layers):
          mamba=Mamba(d_model=hidden_dim, d_state=16, d_conv=4, expand=2)
          mamba.to(device)
          self.mamba_layers.append(mamba)
        # uncomment below when you have a good GPU
        # self.conv1 = GCNConv(num_features, hidden_dim)
        # self.fc1 = nn.Linear(hidden_dim, hidden_dim)

        self.fc1 = nn.Linear(num_features, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, sequence, inclusion, subgraph_size):
        # Preprocessing
        emb = embeddings[sequence]
        last_entry=inclusion.unsqueeze(-1)
        emb = torch.cat((emb, last_entry), dim=-1)

        # start with linear layer projection
        # emb = self.conv1(emb, self.edge_tensor, None)
        emb = self.fc1(emb)

        # Mamba layers
        for mamba in self.mamba_layers:
          emb=mamba(emb)

        # a bit awkward but I don't know how else to compute my aggregation
        agg = []
        for e, size in zip(emb, subgraph_size):
          agg.append(torch.sum(e[-size:], axis=0)/size)
        emb = torch.stack(agg)

        # Linear layer for final class, assume 0-1 classification
        emb = self.fc2(emb)
        emb = F.sigmoid(emb)

        return emb

## Checking Model

In [ ]:
embeddings = embeddings.to(device)
edge_tensor = edge_tensor.to(device)
train_dataset = SubgraphDataset(train_sub_G, train_sub_G_label, degrees, neighbor_dict)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_dataset = SubgraphDataset(val_sub_G, val_sub_G_label, degrees, neighbor_dict)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=True)

In [ ]:
sequence, inclusion, subgraph_size, _ = next(iter(train_loader))
model = SubgraphMamba(129, 5, 5, 2, embeddings, num_mamba_layers=2)

embeddings = embeddings.to(device)
sequence = sequence.to(device)
inclusion = inclusion.to(device)
subgraph_size = subgraph_size.to(device)
model.to(device)

In [ ]:
model(sequence, inclusion, subgraph_size)

# Training and Testing

In [ ]:
def train():
    max_acc = 0.00
    patience_cnt = 0
    val_acc_values = []
    best_epoch = 0

    t = time()
    model.train()
    for epoch in range(5):
        loss_train = 0.0
        correct = 0
        for i, data in enumerate(train_loader):
            sequence, inclusion, subgraph_size, y = data
            optimizer.zero_grad()
            sequence = sequence.to(device)
            inclusion = inclusion.to(device)
            subgraph_size = subgraph_size.to(device)
            y=y.to(device)
            out = model(sequence, inclusion, subgraph_size)

            loss = criterion(out, y)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            pred = out.max(dim=1)[1]
            correct += pred.eq(y).sum().item()
        acc_train = correct / len(train_loader.dataset)

        acc_val, loss_val = compute_test(val_loader)
        print('Epoch: {:04d}'.format(epoch + 1), 'loss_train: {:.4f}'.format(loss_train),
              'acc_train: {:.4f}'.format(acc_train),
              'acc_val: {:.4f}'.format(acc_val))

        val_acc_values.append(acc_val)
        torch.save(model.state_dict(), '{}.pth'.format(epoch))
        if val_acc_values[-1] > max_acc:
            max_acc = val_acc_values[-1]
            best_epoch = epoch
            patience_cnt = 0
        else:
            patience_cnt += 1

        if patience_cnt == 100:
            break

        files = glob.glob('*.pth')
        for f in files:
            epoch_nb = int(f.split('.')[0])
            if epoch_nb < best_epoch:
                os.remove(f)

    files = glob.glob('*.pth')
    for f in files:
        epoch_nb = int(f.split('.')[0])
        if epoch_nb > best_epoch:
            os.remove(f)
    print('Optimization Finished! Total time elapsed: {:.6f}'.format(time() - t))

    return best_epoch


def compute_test(loader):
    model.eval()
    correct = 0.0
    loss_test = 0.0
    for data in loader:
        sequence, inclusion, subgraph_size, y = data
        sequence = sequence.to(device)
        inclusion = inclusion.to(device)
        subgraph_size = subgraph_size.to(device)
        y=y.to(device)
        out = model(sequence, inclusion, subgraph_size)
        pred = out.max(dim=1)[1]
        correct += pred.eq(y).sum().item()
        loss_test += criterion(out, y).item()
    return correct / len(loader.dataset), loss_test

In [ ]:
embeddings = embeddings.to(device)
# edge_tensor = edge_tensor.to(device)
train_dataset = SubgraphDataset(train_sub_G, train_sub_G_label, degrees, neighbor_dict)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_dataset = SubgraphDataset(val_sub_G, val_sub_G_label, degrees, neighbor_dict)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=True)

In [ ]:
model = SubgraphMamba(129, 32, 32, 2, embeddings, num_mamba_layers=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay = 0.01)
criterion = nn.CrossEntropyLoss()
model.to(device)

SubgraphMamba(
  (mamba): Mamba(
    (in_proj): Linear(in_features=16, out_features=64, bias=False)
    (conv1d): Conv1d(32, 32, kernel_size=(4,), stride=(1,), padding=(3,), groups=32)
    (act): SiLU()
    (x_proj): Linear(in_features=32, out_features=33, bias=False)
    (dt_proj): Linear(in_features=1, out_features=32, bias=True)
    (out_proj): Linear(in_features=32, out_features=16, bias=False)
  )
  (fc1): Linear(in_features=129, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=2, bias=True)
)

In [ ]:
best_model=train()

Epoch: 0001 loss_train: 15.9594 acc_train: 0.4912 acc_val: 0.4490
Epoch: 0002 loss_train: 15.9507 acc_train: 0.4912 acc_val: 0.4490
Epoch: 0003 loss_train: 15.9534 acc_train: 0.4912 acc_val: 0.4490
Epoch: 0004 loss_train: 15.9594 acc_train: 0.4912 acc_val: 0.4490
Epoch: 0005 loss_train: 15.9557 acc_train: 0.4912 acc_val: 0.4490
Optimization Finished! Total time elapsed: 20.979321


In [ ]:
best_model = train()
    # Restore best model for test set
model.load_state_dict(torch.load('{}.pth'.format(best_model)))
test_acc, test_loss = compute_test(test_loader)
print('Test set results, loss = {:.6f}, accuracy = {:.6f}'.format(test_loss, test_acc))

Epoch: 0001 loss_train: 19.1891 acc_train: 0.6011 acc_val: 0.6216
Epoch: 0002 loss_train: 18.3392 acc_train: 0.6169 acc_val: 0.6486
Epoch: 0003 loss_train: 17.5171 acc_train: 0.6360 acc_val: 0.6577
Epoch: 0004 loss_train: 17.0323 acc_train: 0.6652 acc_val: 0.6847
Epoch: 0005 loss_train: 17.0640 acc_train: 0.6708 acc_val: 0.6937
Epoch: 0006 loss_train: 16.8557 acc_train: 0.6910 acc_val: 0.6847
Epoch: 0007 loss_train: 16.6482 acc_train: 0.6955 acc_val: 0.7117
Epoch: 0008 loss_train: 16.3526 acc_train: 0.7146 acc_val: 0.7117
Epoch: 0009 loss_train: 16.3752 acc_train: 0.7124 acc_val: 0.7207
Epoch: 0010 loss_train: 16.3334 acc_train: 0.7146 acc_val: 0.7207
Epoch: 0011 loss_train: 15.9987 acc_train: 0.7292 acc_val: 0.7658
Epoch: 0012 loss_train: 15.8299 acc_train: 0.7258 acc_val: 0.7928
Epoch: 0013 loss_train: 15.7283 acc_train: 0.7326 acc_val: 0.7838
Epoch: 0014 loss_train: 15.7212 acc_train: 0.7427 acc_val: 0.7207
Epoch: 0015 loss_train: 15.8624 acc_train: 0.7303 acc_val: 0.7568
Epoch: 001